# Generacion de un archivo de entrega a Kaggle

In [18]:
import numpy as np
import pandas as pd
import Utilidades as ut
import keras
import tensorflow as tf

### Definicion de funciones generales

In [19]:
def predict_and_save(model, x, df_entrega, path):
    y_pred = model.predict(x).flatten()
    #Armamos el df de entrega
    df_entrega['Target'] = pd.Series(y_pred)
    df_entrega = df_entrega.groupby(['Opportunity_ID']).agg({'Target' : 'mean'}).reset_index()
    df_entrega['Target'] = (df_entrega['Target'] > 0.5).astype(int)
    #Guardamos en un csv
    df_entrega.to_csv(path, index=False)

### Preparacion de los datos

In [20]:
x_test = pd.read_pickle("Archivos/Neuronales_test.pkl")
x_train = pd.read_pickle("Archivos/Neuronales_entrenamiento.pkl")

x_train, y_train = ut.split_labels(x_train)

df_entrega = x_test[['Opportunity_ID']]
x_test = x_test.drop(columns=['Opportunity_ID'])

#Convertimos las fechas a numeros (cantidad de dias transcurridos) y luego las normalizamos
x_test = ut.conversion_fechas(x_test)
x_train, x_test = ut.codificar_categoricas(x_train, y_train, x_test, modo='catboost')
x_test = ut.normalizacion_numericas(x_test, modo='normalizacion')

#Convertimos los df a vectores
x_vector = ut.df_a_vector(x_test)


/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [21]:
df_entrega.head(3)

,Opportunity_ID
0,11223
1,11223
2,11223


## Modelos

In [22]:
#Lista de los modelos disponibles, reemplazar en la variable de abajo para elegir uno
model_list = ['Neural_Network']

model_name = 'Neural_Network'
path = 'Entrega_Neuronales.csv'

model = None

# Seguramente se puede hacer de otra forma mas 'python'

if (model_name == 'Neural_Network'):
    model = keras.models.load_model('Redes_Neuronales/Neuronales_Mejor_Modelo.hdf5')

predict_and_save(model, x_vector, df_entrega, path=path)